In [2]:
import csv
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
# driver = webdriver.Edge()
driver = webdriver.Chrome(r'your_driver_path\chromedriver')
driver.get('https://www.amazon.eg/-/en/s?i=fashion&bbn=21845147031&rh=n%3A18018165031%2Cn%3A21845147031%2Cn%3A21845179031&dc&fs=true&page=2&language=en&qid=1675347559&rnid=21845147031&ref=sr_pg_2')

In [5]:
# get the whole div that carrying all the details of the product
wholeDiv = driver.find_elements(By.XPATH,'//div[@class="s-card-container s-overflow-hidden aok-relative puis-expand-height puis-include-content-margin puis s-latency-cf-section s-card-border"]')

In [ ]:
# print properities of a single product
print(wholeDiv[0].get_property('innerHTML'))

In [ ]:
with open('.\File.csv', 'w',encoding='UTF-8',newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    # header of the file
    csvwriter.writerow(['Product Name','Price','Rating','Img Path','Category','Category Name','Description'])
    j = 0


    for k in range(1,10):
        wholeDiv = driver.find_elements(By.XPATH,'//div[@class="s-card-container s-overflow-hidden aok-relative puis-expand-height puis-include-content-margin puis s-latency-cf-section s-card-border"]')

        for i in wholeDiv:
            # rating
            rating = None
            txt = i.get_property('innerHTML')
            x = re.search('<div class="a-row a-size-small"><span aria-label="',txt)
            y = re.search('out of 5 stars',txt)
            if x and y:
                rating = txt[x.span()[1]:y.span()[0]]

            # price
            price = 0
            txt = i.get_property('innerHTML')
            x = re.search('<span class="a-price-whole">',txt)
            y = re.search('<span class="a-price-decimal"',txt)
            
            if x and y:
                price = txt[x.span()[1]:y.span()[0]] 
                price = re.sub(",", "", price)
            
            # name
            txt = i.get_property('innerHTML')
            x = re.search('<span class="a-size-base-plus a-color-base a-text-normal">',txt)
            y = re.search('</span>',txt[x.span()[1]:])

            name = txt[x.span()[1]:y.span()[0]+x.span()[1]]
            
            # image
            txt = i.get_property('innerHTML')
            x = re.search('<div class="a-section aok-relative s-image-tall-aspect"><img class="s-image" src="',txt)
            y = re.search('" srcset="',txt)

            # download images into /images folder
            img_data = requests.get(txt[x.span()[1]:y.span()[0]]).content
            with open(f'images/{j}.jpg', 'wb') as handler:
                handler.write(img_data)
            
            # write data into csv file
            csvwriter.writerow([name,float(price),rating,f'images/{j}.jpg','Men','Accessories',name])

            j += 1
        print(f"Page {k} done")
        # link of the next page with update of k
        driver.get(f'https://www.amazon.eg/-/en/s?i=fashion&bbn=21845147031&rh=n%3A18018165031%2Cn%3A21845147031%2Cn%3A21845179031&dc&fs=true&page={k+1}&language=en&qid=1675347559&rnid=21845147031&ref=sr_pg_2')
        
